<a href="https://colab.research.google.com/github/JairamDesik/analyzing-weather-dataset/blob/master/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TASK 1: Date Manipulation**

In [ ]:
# Here, I am creating a dataframe where there are four date columns with different format date values in each of them 

import pandas as pd
import numpy as np
from datetime import datetime
Date_Time = ['01/02/2019 00:30:00', '29/12/2019', '20/12/2019 20:00:02', np.nan, '12/01/2020 00:00:00', '2020/2/20']
codes = [201, 202, 203, 204, 205, 206]
date1 = ['4/5/2019', '25/6/2019', np.nan, '15/05/2019 10:00:25', 'Saturday, 6, June, 20', '25/4/2020']
names = ['Ajay', 'Ram', 'Raj', 'Krish', 'Sid', 'Haeish']
date2 = ['2019/6/2', np.nan, '2019/6/29', 'Thursday, 21, March, 19', '30/03/2020 00:20:10', '2020/5/1']
date3 = ['Wednesday, 23, January, 19', '2019/10/30', 'Friday, 30, August, 19', 'Monday, 17, February, 20', np.nan, '30/1/2020' ]
marks = [85, 89, 95, 99, 91, 88]
roll_no = ['P0211', 'P0212', 'P0206', 'U0208', 'U0206', 'U0209']
first = pd.DataFrame({'Date/Time' : Date_Time, 'Code' : codes, 'Date_1' : date1, 'Name' : names, 'Roll_No' : roll_no, 'Date_2' : date2, 'Marks' : marks, 'Date3': date3})
first

,Date/Time,Code,Date_1,Name,Roll_No,Date_2,Marks,Date3
0,01/02/2019 00:30:00,201,4/5/2019,Ajay,P0211,2019/6/2,85,"Wednesday, 23, January, 19"
1,29/12/2019,202,25/6/2019,Ram,P0212,NaN,89,2019/10/30
2,20/12/2019 20:00:02,203,NaN,Raj,P0206,2019/6/29,95,"Friday, 30, August, 19"
3,NaN,204,15/05/2019 10:00:25,Krish,U0208,"Thursday, 21, March, 19",99,"Monday, 17, February, 20"
4,12/01/2020 00:00:00,205,"Saturday, 6, June, 20",Sid,U0206,30/03/2020 00:20:10,91,NaN
5,2020/2/20,206,25/4/2020,Haeish,U0209,2020/5/1,88,30/1/2020


In [ ]:
# The function for date manipulation

def date_manipulation(df, diff_in = 'days'):
    """ Extract date columns and get new columns from them by performimg Subtraction
    
    This function accepts a dataframe and returns new dataframe,
    containing all the date columns made to standard format (i.e., YYYY-MM-DD)
    and new columns are added which are formed by subtracting two date columns to get
    duration in days, weeks, months and years. But the defult duration will be days.
    
    Keyword arguments:
    df - Pandas dataframe from which the columns name will be extracted
    diff_in - A string specifying the way in which the duration must be represented
        
    Returns:
    df - Pandas datafrome with dates being converted to a stabdard format (DD-MM-YYYY)
         and new columns are added which are difference of two date columns
    """
    # Initially we separate categorical columns from the total DataFrame
    categorical_var= df.select_dtypes(include='object').columns.tolist()
    print(categorical_var)

    df_cat = df[categorical_var].copy()
    date_col = []
    # We need to convert date columns info to a standard format for making it useful for further analysis
    for i in categorical_var:
        df_cat[i] = pd.to_datetime(df_cat[i], errors='coerce').dt.strftime('%Y-%m-%d')
        if df_cat[i].dtype == 'object':# All the Non-date column values will be NaN with dtype = 'float'
            df[i] = df_cat[i]          # Now we can copy changes in Date columns to original DataFrame
            date_col.append(i)
            df[i] = pd.to_datetime(df[i], errors='coerce') # This is done to make the dtype of Date columns to
                                                           # datetime[ns], so that we can do elementary opporations 
                                                           # like Subtraction, Addition, etc on Date columns

    print(date_col)
    
    # Finally, we need to perform Subtraction on two different Date columns to get new Duration column, which can be
    # used for further predictive analysis
    for i,k in enumerate(date_col):
        for j in date_col[i+1:]:
            df[k +' - '+ j] = df[k] - df[j]
            if diff_in == 'weeks':
                df[k +' - '+ j] = df[k +' - '+ j]/np.timedelta64(1, 'W')
            elif diff_in == 'months':
                df[k +' - '+ j] = df[k +' - '+ j]/np.timedelta64(1, 'M')
            elif diff_in == 'years':
                df[k +' - '+ j] = df[k +' - '+ j]/np.timedelta64(1, 'Y')

    return df

In [ ]:
# Checking the output

first_out = date_manipulation(first, diff_in = 'weeks')
first_out

['Date/Time', 'Date_1', 'Name', 'Roll_No', 'Date_2', 'Date3']
['Date/Time', 'Date_1', 'Date_2', 'Date3']


,Date/Time,Code,Date_1,Name,Roll_No,Date_2,Marks,Date3,Date/Time - Date_1,Date/Time - Date_2,Date/Time - Date3,Date_1 - Date_2,Date_1 - Date3,Date_2 - Date3
0,2019-01-02,201,2019-04-05,Ajay,P0211,2019-06-02,85,2019-01-23,-13.285714,-21.571429,-3.000000,-8.285714,10.285714,18.571429
1,2019-12-29,202,2019-06-25,Ram,P0212,NaT,89,2019-10-30,26.714286,NaN,8.571429,NaN,-18.142857,NaN
2,2019-12-20,203,NaT,Raj,P0206,2019-06-29,95,2019-08-30,NaN,24.857143,16.000000,NaN,NaN,-8.857143
3,NaT,204,2019-05-15,Krish,U0208,2019-03-21,99,2020-02-17,NaN,NaN,NaN,7.857143,-39.714286,-47.571429
4,2020-12-01,205,2020-06-06,Sid,U0206,2020-03-30,91,NaT,25.428571,35.142857,NaN,9.714286,NaN,NaN
5,2020-02-20,206,2020-04-25,Haeish,U0209,2020-05-01,88,2020-01-30,-9.285714,-10.142857,3.000000,-0.857143,12.285714,13.142857


# **TASK 2: Function to reduce no. of rows with pearrson coeff > 0.85**

In [ ]:
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt
import seaborn as sb
from statsmodels.stats.outliers_influence import variance_inflation_factor

def correlation(dataset, threshold, method='Multicorr'):
    """ Take a dataset, perform pearson's correlation and outputs dataset with reduced no. of efficient features

    This function accepts dataset and performs pearson's correlation on the features,
    delets the columns with pearson correlation coeff. > 0.85 and the deletion of column is done 
    by either considering correlation with target or by considering multicorrelation with all features
    as per the requirement of user

    Keyword arguments:
    dataset: Pandas dataframe on which feature engineering is done
    threshold: the pearson's correlation coefficient, which is used to set as threshold for elimination od column
    method: This is used to select the method of elimination of one of the two columns whose pearson coeff > threshold

    Returns:
    dataset: Pandas dataframe on which feature engineering based on pearsons correlation is done
    """ 

    col_corr = set() # Set of all the names of deleted columns

    for i in range(len(dataset.corr().columns)):
        corr_matrix = dataset.corr()
        for j in range(i):
            if (corr_matrix.iloc[i, j] >= threshold) and (corr_matrix.iloc[i,j]<1) and (corr_matrix.columns[j] not in col_corr):
            # selection of elimination method as per the user
                if method == 'Pairwise': # The column with lowest correlation w.r.t target is removed 
                    if corr_matrix.iloc[:, -1:].iloc[i] > corr_matrix.iloc[:, -1:].iloc[j]:
                        colname = corr_matrix.columns[i] # getting the name of column
                        col_delete(colname)
                    else:
                        colname = corr_matrix.columns[i] # getting the name of column
                        col_delete(colname)
                elif method == 'Multicorr': # The column with highest VIF will be removed
                    if calc_vif(dataset, i) > calc_vif(dataset, j):
                        colname = corr_matrix.columns[i] # getting the name of column
                        col_delete(colname)
                    else:
                        colname = corr_matrix.columns[j]
                        col_delete(colname)

    def calc_vif(X, int):
        # Calculating VIF
        vif = pd.DataFrame()
        vif["variables"] = X.columns
        vif["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
        return(vif['VIF'][int])

    def col_delete(colname):
        col_corr.add(colname)
        if colname in dataset.columns:
            del dataset[colname] # deleting the column from the dataset

    # Good Visualization is requierd to get proper understanding
    sb.heatmap(corr_matrix, 
            xticklabels=corr_matrix.columns,
            yticklabels=corr_matrix.columns,
            cmap='RdBu_r',
            annot=True,
            linewidth=0.5)
    
    return(dataset)

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
